#### Display

In [30]:
import psycopg2
import pandas as pd
#from wine_quality.inference import make_predictions

try:
    # Connect to PostgreSQL database
    connection = psycopg2.connect(
        user="postgres",
        password="dsp123456",
        host="localhost",
        port="5432",
        database="red_wine_data"
    )

    # create a cursor
    cursor = connection.cursor()

    # execute SQL cursor
    cursor.execute("SELECT * from past_wine_data;")
    # fetch cursor record
    record = cursor.fetchone()
    print("You are connected to - ", record, "\n")

except (Exception, psycopg2.Error) as error:
    print("Error while connecting to PostgreSQL", error)

finally:
    # close connection
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")


You are connected to -  ('a',) 

PostgreSQL connection is closed


#### Create Tables

In [18]:
try:
    # Connect to PostgreSQL database
    connection = psycopg2.connect(
        user="postgres",
        password="dsp123456",
        host="localhost",
        port="5432",
        database="red_wine_data"
    )

    # create a cursor
    cursor = connection.cursor()

    # execute SQL cursor
    cursor.execute("create table past_wine_data (id varchar(20),fixed_acidity float,volatile_acidity float,citric_acid float,residual_sugar float,chlorides float,free_sulfur_dioxide float,total_sulfur_dioxide float,density float,pH float,sulphates float,alcohol float);")
    connection.commit()
    cursor.execute('''create table prediction (
                        wine_id varchar(20),
                        predictions float)''')
    connection.commit()
    cursor.execute('''create table errors (
                        wine_id varchar(20),
                        errors varchar(20))''')   
    connection.commit()
except (Exception, psycopg2.Error) as error:
    print("Error while connecting to PostgreSQL", error)

finally:
    # close connection
    if connection:
        
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")


PostgreSQL connection is closed


#### Load Dataset

In [39]:
try:
    connection = psycopg2.connect(
        user="postgres",
        password="dsp123456",
        host="localhost",
        port="5432",
        database="red_wine_data"
    )
    data = pd.read_csv('C:/Users/606/Downloads/wine/winequality-red.csv')
    for index, row in data.iterrows():
        id = index
        fixed_acidity = row['fixed acidity']
        volatile_acidity = row['volatile acidity']
        citric_acid = row['citric acid']
        residual_sugar = row['residual sugar']
        chlorides = row['chlorides']
        free_sulfur_dioxide = row['free sulfur dioxide']
        total_sulfur_dioxide = row['total sulfur dioxide']
        density = row['density']
        pH = row['pH']
        sulphates = row['sulphates']
        alcohol = row['alcohol']
        # create a cursor
        cursor = connection.cursor()
        # execute SQL cursor
        sql='''
             insert into past_wine_data (
             id,
             fixed_acidity,
             volatile_acidity,
             citric_acid,
             residual_sugar,
             chlorides,
             free_sulfur_dioxide,
             total_sulfur_dioxide,
             density,
             pH,
             sulphates,
             alcohol) values
             ('''+str(id)+','+str(fixed_acidity)+','+str(volatile_acidity)+','+str(citric_acid)+','+str(residual_sugar)+','+str(chlorides)+','+str(free_sulfur_dioxide)+','+str(total_sulfur_dioxide)+','+str(density)+','+str(pH)+','+str(sulphates)+','+str(alcohol)+')'
        
        
        cursor.execute(sql)
        connection.commit()

except (Exception, psycopg2.Error) as error:
    print("Error while connecting to PostgreSQL", error)

finally:
    # close connection
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

PostgreSQL connection is closed
